In [1]:
#Import Packages
import numpy as np
import os
import pickle
from multicontact_learning_local_objectives.python.machine_learning.ml_utils import *
import matplotlib.pyplot as plt #Matplotlib
import shutil

print("Double check the Path for storing trajectories is correct")

Double check the Path for storing trajectories is correct


In [2]:
#Define Path for Storing Trajectories
#Collect Data Points Path
#workingDirectory = "/home/jiayu/Desktop/multicontact_learning_local_objectives/data/large_slope_flat_patches/"
#workingDirectory = "/home/jiayu/Desktop/MLP_DataSet/Rubbles"
#workingDirectory = "/home/jiayu/Desktop/MLP_DataSet/Rubbles_Standarized_Data/"
workingDirectory = "/home/jiayu/Desktop/MLP_DataSet/Rubbles/"
#NOTE: need to have "/" at the end
print("Double Check we provide the Correct Traj Path: \n", workingDirectory)

#Define dataset folder
#DataSetPath = workingDirectory + "/DataSet/" + "Standarization"
#DataSetPath = workingDirectory + "/DataSet_OriginalForm"
DataSetPath = workingDirectory + "/DataSet/"+"MaxAbs"

#Path to store ML Model, create one if we dont have
ML_Model_Path = workingDirectory + "/ML_Models/"
if not (os.path.isdir(ML_Model_Path)):
    os.mkdir(ML_Model_Path)


Double Check we provide the Correct Traj Path: 
 /home/jiayu/Desktop/MLP_DataSet/Rubbles/


In [3]:
#Learning Code
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.neighbors import KNeighborsRegressor

In [4]:
#Load DataSet File

#dataset_file = DataSetPath + "/data_rubbles"+'.p'
dataset_file = DataSetPath + "/data"+'.p'
dataset = pickle.load(open(dataset_file,"rb"))

print("World Frame Shift: ", dataset["Shift_World_Frame_Type"])
print("Contact Location Representation Type: ",dataset["Contact_Representation_Type"])
print("Scaling Factor of Variables: ",dataset["VectorScaleFactor"])
print("Number of Preview Steps: ", dataset["NumPreviewSteps"])
print("Pre Process Mode: ",dataset["PreProcessMode"])

World Frame Shift:  StanceFoot
Contact Location Representation Type:  FollowRectangelBorder
Scaling Factor of Variables:  1
Number of Preview Steps:  4
Pre Process Mode:  MaxAbs


In [5]:
#Test Train Split, for now No Test data
#x_train, x_test, y_train, y_test = train_test_split(dataset["input"], dataset["output"], test_size = 0.01)
x_train = dataset["input"]
y_train = dataset["output"]

#Get Left Foot to see
#x_train=x_train[::2]
#y_train=y_train[::2]


#Decide input and outpu dimensionality
d_in = x_train[0].shape[0]
print("input dim: ", d_in)
d_out = y_train[0].shape[0]
print("output dim:", d_out)
print(" ")

#Double check with mean and std
print("Inputs: ")
print("Input Mean: ", x_train.mean(axis=0))
print("Input Std: ", x_train.std(axis=0))
print("Input Max: ", x_train.max(axis=0))
print("Input Min: ", x_train.min(axis=0))
print(" ")


print("Output Mean: ", y_train.mean(axis=0))
print("Output Std: ", y_train.std(axis=0))
print("Output Max: ", y_train.max(axis=0))
print("Output Min: ", y_train.min(axis=0))

input dim:  85
output dim: 11
 
Inputs: 
Input Mean:  [-4.02629463e-01  7.15567103e-03  9.15109496e-01  3.89550847e-01
  1.12374648e-02 -5.07501980e-03 -1.95048670e-03 -3.74361555e-03
  7.21433185e-04 -8.38346728e-01  1.50332575e-04  9.70247892e-03
  5.00000000e-01  5.92445028e-01  7.15301040e-01 -1.56373248e-04
 -5.94033041e-01  7.18817409e-01 -4.30810553e-04 -5.95708509e-01
  2.15570654e-03  6.74540480e-04  5.92459967e-01  2.20184929e-03
  1.22248936e-03 -2.14242343e-02  1.15694583e-02 -1.47883834e-03
 -7.10574410e-01  1.15578640e-02 -7.04677589e-04 -7.10535301e-01
 -7.07444509e-01 -1.35142539e-03 -2.12846786e-02 -7.07440929e-01
 -1.61305320e-03  7.31356222e-01  3.61740161e-01 -6.31418802e-04
 -2.51307425e-02  3.58108689e-01 -6.20403407e-04 -2.45435838e-02
 -3.50718068e-01 -4.35274697e-04  7.31287734e-01 -3.54072558e-01
 -4.46359275e-04  8.18043218e-01  5.22779909e-01  5.65523008e-04
  5.89772902e-01  5.22788284e-01  6.07401896e-04  5.89689516e-01
 -5.18185670e-01  9.01942072e-04  8.

In [6]:
#Build kNN
K = 5 #number of neighbors
knn = KNeighborsRegressor(K)

#fit kNN
knn.fit(x_train, y_train)

KNeighborsRegressor()

In [7]:
#Show Prediction Result for Training
from sklearn import preprocessing

datapoint_num = 2
y_pred_train = knn.predict((np.array([x_train[datapoint_num]])))

print(x_train[datapoint_num])

#Recover to original format
if dataset["PreProcessMode"] == "OriginalForm":
    print("Data Kept Original Form, No need to transform back")
    pred_y_originalform = y_pred_train
    true_y_originalform = y_train[datapoint_num]
elif dataset["PreProcessMode"] == "Standarization" or dataset["PreProcessMode"] == "MaxAbs":
    pred_y_originalform = dataset["Scaler_Y"].inverse_transform(y_pred_train)
    true_y_originalform = dataset["Scaler_Y"].inverse_transform(np.array([y_train[datapoint_num]]))
else:
    raise Exception("Unknow Pre Process Mode")


print("predicted result: \n",pred_y_originalform)
print("true value: \n",true_y_originalform)
print("diff: \n", np.absolute(pred_y_originalform - true_y_originalform))

[-4.01157749e-01  6.81034247e-01  9.31168669e-01  4.53803804e-01
 -2.52448386e-01  7.23417586e-02  1.04711521e-06 -1.05721493e-07
 -4.05414244e-07 -9.56070690e-01  3.17949716e-01 -3.06829343e-01
  1.00000000e+00  2.73399906e-01  7.51042313e-01 -3.57547705e-01
 -9.19070116e-01  7.54749897e-01 -3.57547705e-01 -9.21685103e-01
  1.24612220e-01 -2.37963110e-01  2.73399906e-01  1.24612220e-01
 -2.37963110e-01  5.80010991e-01  1.38185854e-01 -1.79635940e-13
 -5.56672536e-01  1.38185854e-01 -1.32562773e-13 -5.56672536e-01
 -6.71565056e-01 -8.97374935e-14  5.80010991e-01 -6.71565056e-01
 -8.97374935e-14  9.09613652e-01  7.58811374e-01 -3.55636615e-01
  5.55848055e-01  7.51166687e-01 -3.55636615e-01  5.38000731e-01
  3.93869174e-02 -1.19290075e-01  9.09613652e-01  3.97621884e-02
 -1.19290075e-01  6.75976280e-01  5.29281028e-02 -3.64723503e-01
  2.73303809e-01  6.30601585e-02  2.30783300e-02  2.73303809e-01
 -9.78980504e-01  4.97776347e-01  6.75976280e-01 -9.89145035e-01
 -1.69538667e-02  9.48111

In [ ]:
import matplotlib.pyplot as plt

y_pred = knn.predict(x_train)


if dataset["PreProcessMode"] == "OriginalForm":
    print("Data Kept Original Form, No need to transform back")
    y_pred_originalform = y_pred
    y_true_originalform = y_train
elif dataset["PreProcessMode"] == "Standarization" or dataset["PreProcessMode"] == "MaxAbs":
    print("PreProcessing of: ", dataset["PreProcessMode"])
    y_pred_originalform = dataset["Scaler_Y"].inverse_transform(y_pred)
    y_true_originalform = dataset["Scaler_Y"].inverse_transform(y_train)
else:
    raise Exception("Unknow Pre Process Mode")

#Compute Error
err = np.linalg.norm(y_true_originalform-y_pred_originalform, axis=1)

#Plot Histogram
fig=plt.figure();   ax = fig.gca()
plt.hist(err, bins=50, density = True, range = (0.0, 0.375))
ax.set_xlabel("Normalised Error")
ax.set_xlim([-0.025,0.375])
ax.set_ylabel("Percentage")
ax.set_ylim([-1,50])

#### Sort the error

err_sorted = np.sort(err)
print(err_sorted[-300:])  # print the 100 biggest error

print("Error Mean: ", err_sorted.mean())
print("Error Std", err_sorted.std())